In [41]:
import bppy as bp
from bppy.model.sync_statement import *
from bppy.model.b_thread import *
import numpy as np
np.set_printoptions(legacy='1.25')

In [2]:
comps = 'BUSBAR GRID CB_UP_1 CB_UP_2 transfo1 transfo2 CB_DW_1 CB_DW_2 diesel_gen CB_dies'
comps = comps.split(' ')
comps

['BUSBAR',
 'GRID',
 'CB_UP_1',
 'CB_UP_2',
 'transfo1',
 'transfo2',
 'CB_DW_1',
 'CB_DW_2',
 'diesel_gen',
 'CB_dies']

In [4]:
in_oper_f_r = 10e-4 # per hour (exp)
on_demand_f_r = 10e-3 # per attempt
repair_rate = 10-1 # per hour (exp)

In [17]:
ON = True
OFF = False

In [3]:
from scipy.stats import expon

In [9]:
a = expon.rvs(scale=1/in_oper_f_r, size=10, random_state=0)
a
# time to err from start

array([ 795.87450816, 1255.93076297,  923.2231458 ,  787.20115232,
        551.0484911 , 1038.15929494,  575.51919917, 2223.52441303,
       3314.91218205,  483.6021009 ])

In [14]:
b = expon.rvs(loc=0, scale=1/in_oper_f_r, size=10, random_state=0)
b

array([ 795.87450816, 1255.93076297,  923.2231458 ,  787.20115232,
        551.0484911 , 1038.15929494,  575.51919917, 2223.52441303,
       3314.91218205,  483.6021009 ])

In [22]:
b.argmin()

np.int64(9)

In [19]:
sum({'a':ON}.values())

1

In [39]:
3.0

3.0

In [43]:
import matplotlib.pyplot as plt

ModuleNotFoundError: No module named 'matplotlib'

In [42]:
t = 0
fails = {0: [],1: []}
while t < 1e3:
    in_oper = 2
    next_fails = expon.rvs(scale=1/10e-2, size=in_oper)
    c = next_fails.argmin()
    fails[c].append(round(t+next_fails[c], 2))
    t += next_fails[c]
print(fails)

{0: [4.87, 10.14, 12.45, 37.07, 49.72, 55.36, 69.01, 74.01, 75.78, 76.48, 76.56, 78.24, 84.11, 115.38, 128.32, 132.68, 139.02, 167.23, 168.76, 187.26, 206.29, 209.39, 213.48, 224.7, 236.18, 243.63, 264.46, 266.32, 286.42, 302.16, 306.68, 312.96, 326.83, 333.62, 337.33, 361.54, 373.15, 380.37, 381.54, 383.84, 403.31, 411.55, 422.17, 426.84, 426.92, 444.03, 459.07, 471.18, 490.74, 505.56, 530.47, 532.64, 537.82, 544.63, 551.46, 565.05, 581.81, 585.08, 596.03, 602.02, 608.35, 613.04, 619.3, 620.39, 650.07, 670.9, 698.33, 708.8, 743.87, 746.0, 757.29, 765.0, 775.07, 779.45, 793.55, 795.09, 795.7, 796.92, 818.57, 823.1, 842.55, 851.29, 852.57, 865.01, 892.13, 897.04, 899.81, 903.67, 908.65, 910.13, 920.36, 927.98, 933.02, 943.8, 951.13, 984.39, 1004.88], 1: [2.65, 47.33, 49.07, 49.18, 50.25, 53.55, 63.37, 72.97, 103.99, 165.28, 166.72, 169.83, 170.92, 182.85, 228.89, 235.99, 260.32, 274.15, 284.58, 299.68, 300.23, 311.36, 319.52, 321.54, 323.44, 327.07, 344.71, 368.98, 385.76, 387.48, 403.4

In [ ]:
@bp.thread
def main():
    comps = {'a': ON, 'b':ON}
    t = 0
    while t < 1e6:
        in_oper = sum(comps.values())
        next_fails = expon.rvs(scale=1/10, size=in_oper)
        c = next_fails.argmin()
        
@bp.thread
def a():
    sync(waitFor=bp.BEvent('fail_a'))

@bp.thread
def b():
    sync(waitFor=bp.BEvent('fail_b'))

In [17]:
@bp.thread
def cycle_counter(max_t = 20):
    fails = []
    for i in range(max_t):
        sync(request=bp.BEvent('tick'))

@bp.thread
def main_grid():
    grid = ON
    while True:
        e = sync(waitFor=[bp.BEvent(i) for i in ['tick']])
        if e.name == 'tick':
            P_failiure_in_1 = in_oper_f_r*math.e**(-in_oper_f_r) # P[Failiure <= 1]
            status = choice({OFF: P_failiure_in_1, ON: 1-P_failiure_in_1})
            if status != grid:
                sync(bp.BEvent(f'GRID_{status}'))
                grid = status

@bp.thread
def line1():
    cs = {comps[i]: ON for i in [2, 4, 6]} #UP, DW, transfo


@bp.thread
def line2():
    cs = {comps[i]: ON if i != 7 else OFF for i in [3, 5, 7]}

@bp.thread
def backup():
    cs = {comps[i]: OFF for i in [-1, -2]}

@bp.thread
def busbar():
    sources = {'LINE1': ON, 'LINE2': OFF, 'DIESEL': OFF}
    is_working = lambda: any(sources.values())
    bb = is_working()


In [7]:
any({'a':0}.values())

False